In [3]:
pip install -U tensorflow_addons

In [4]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir ("/content/drive/MyDrive/FourthBrain/Capstone Samsung OCT/Data/OCT2017")

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np

In [6]:
train_dir = os.getcwd() + "/train/"
valid_dir = os.getcwd() + "/val/"
test_dir = os.getcwd() + "/test/"

In [7]:
classes = ['NORMAL', 'CNV', 'DME', 'DRUSEN']
cols = [x.lower() for x in classes]
dirs = [train_dir, valid_dir, test_dir]
label = {0: 'normal', 1: 'cnv', 2: 'dme', 3: 'drusen'}

In [ ]:
# Printing out the # of samples for each subsample percentage 
print ("Trainig Data percentages:")
print (" 1% ==> ", int(.01 * train_df.shape[0]))
print (" 5% ==> ", int(.05 * train_df.shape[0]))
print ("10% ==> ", int(.1  * train_df.shape[0] ))
print ("25% ==> ", int(.25 * train_df.shape[0]))
print ("75% ==> ", int(.75 * train_df.shape[0]))
print ("90% ==> ", int(.9  * train_df.shape[0]))
print ("98% ==> ", int(.98 * train_df.shape[0]))

In [9]:
# determine class weights to feed into neural network during training
def get_classweight():
  # simple version for working with CWD
  nCNV = len([name for name in os.listdir('train/CNV')])
  nDME = len([name for name in os.listdir('train/DME')])
  nDRUSEN = len([name for name in os.listdir('train/DRUSEN')])
  nNORMAL = len([name for name in os.listdir('train/NORMAL')])

  ncount = np.array([nCNV, nDME, nDRUSEN, nNORMAL])
  ntotal = sum(ncount)

  weights = ntotal/ncount/ len(ncount)
  weights /= weights.min()
  class_weight = {i: w for i, w in enumerate(weights)}  
  return class_weight

In [12]:
class_weight = get_classweight()
class_weight

OSError: ignored

In [11]:
os.getcwd()

'/content/drive/My Drive/FourthBrain/Capstone Samsung OCT/Data/OCT2017'

In [13]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from data_util import *
import tensorflow_addons as tfa

Total number of parameters: 23.56M


In [14]:
#gs://simclr-checkpoints/simclrv2/finetuned_100pct/r101_2x_sk1/hub
gs_path = "gs://simclr-checkpoints/simclrv2/finetuned_100pct/r50_1x_sk0/hub/"

input = tf.keras.layers.Input((224,224,3))
simclr_layer = hub.KerasLayer(gs_path, trainable=False)
output = tf.keras.layers.Dense(4, activation="softmax")

model = tf.keras.Sequential([input, 
                          simclr_layer,
                          output])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              34277672  
_________________________________________________________________
dense (Dense)                (None, 4)                 8196      
Total params: 34,285,868
Trainable params: 8,196
Non-trainable params: 34,277,672
_________________________________________________________________


In [15]:
trn_img = keras.preprocessing.image_dataset_from_directory(train_dir, labels="inferred", color_mode="rgb", batch_size=32,
                                                           image_size=(224, 224), shuffle=True, seed=10, validation_split=0.5,
                                                           subset='training', interpolation="bilinear", follow_links=False)
val_img = keras.preprocessing.image_dataset_from_directory(train_dir, labels="inferred", color_mode="rgb", batch_size=32,
                                                           image_size=(224, 224), shuffle=True, seed=10, validation_split=.05,
                                                           subset='validation', interpolation="bilinear", follow_links=False)

Found 57169 files belonging to 4 classes.
Using 28585 files for training.
Found 83484 files belonging to 4 classes.
Using 4174 files for validation.


In [16]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [17]:
# Creating a checkpoint to save the best model so that we can reload it once training is complete
checkpoint_cb = keras.callbacks.ModelCheckpoint("oct_simclrtrain.h5", save_best_only=True)
# Adding an an early stop callback to avoid overfitting in case the model is not improving after 5 consescutive epochs
earlystop_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
#history = model.fit(trn_img, steps_per_epoch=15, epochs=100, verbose=1, validation_data=val_img, class_weight=class_weight, callbacks=[checkpoint_cb, earlystop_cb])

In [33]:
def get_features(dataset, max_iter=100):
  out_feature, out_label = [], []
  count = 0
  
  for img, lbl in enumerate (dataset):
    out_feature.append(simclr_layer(img).numpy())
    out_label.append(lbl.numpy())
    if count % 100 == 0: print(f'count = {count}')
    if count > max_iter:
      break
    count += 1
  
  out_label = np.hstack(out_label)
  out_feature   = np.vstack(out_feature)
  return out_feature, out_label

In [34]:
valid_feature, valid_label = get_features(val_img)

RuntimeError: ignored

In [20]:
type(val_img)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [21]:
def get_representation(dataset, max_batch=500):
  out_rep, out_label = [], []
  count = 0
  for img, lb in dataset:
    out_rep.append(simclr_layer(img).numpy())
    out_label.append(lb.numpy())
    if count % 100 == 0: print(f'count = {count}')
    if count > max_batch: break
    count += 1
  out_label = np.hstack(out_label)
  out_rep   = np.vstack(out_rep)
  return out_rep, out_label

In [22]:
train_rep, train_lb = get_representation(trn_img)

RuntimeError: ignored